In [34]:
import pyreadr
import pandas as pd
rRead = pyreadr.read_r('exgr_test.rds')

In [35]:
df_python = pd.DataFrame(rRead[None])
df_python

,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank
0,1,ENST00000456328.2,chr1,11869,12227,359,+,1,ENSE00002234944.1,1
1,1,ENST00000456328.2,chr1,12613,12721,109,+,5,ENSE00003582793.1,2
2,1,ENST00000456328.2,chr1,13221,14409,1189,+,8,ENSE00002312635.1,3
3,2,ENST00000450305.2,chr1,12010,12057,48,+,2,ENSE00001948541.1,1
4,2,ENST00000450305.2,chr1,12179,12227,49,+,3,ENSE00001671638.2,2
...,...,...,...,...,...,...,...,...,...,...
1460981,234481,ENST00000387409.1,chrM,5826,5891,66,-,760545,ENSE00001544488.1,1
1460982,234482,ENST00000387416.2,chrM,7446,7514,69,-,760546,ENSE00001544487.2,1
1460983,234483,ENST00000361681.2,chrM,14149,14673,525,-,760547,ENSE00001434974.2,1
1460984,234484,ENST00000387459.1,chrM,14674,14742,69,-,760548,ENSE00001544476.1,1


Task 1: How many unique transcripts are there? 

In [36]:
df_python['transcript_id'].nunique()

234485

Task 2: How many unique Exons are there?

In [37]:
df_python['exon_id'].nunique()

760549

Task 3: What is the average length of an exon? What is the median length?

In [38]:
# Average length of exon:
df_python['width'].mean()

262.9930635885628

In [39]:
# Median length of exon:
df_python['width'].median()

130.0

Task 4: Calculate the length of intron

In [40]:
# We don't know if the data set was grouped and ordered by transcript_id and rank, so do it before operation
df_python = df_python.sort_values(by=['transcript_id', 'rank'])

In [41]:
# split the dataset into 2 sets, based on strand 
positive_strand = df_python['strand'] == '+'
negative_strand = df_python['strand'] == '-'


In [42]:
# Calculate length of intron
# for positive strand: intron_length = nth start - (n-1)th end
df_python.loc[positive_strand, 'intron_length'] = df_python.loc[positive_strand, 'start'] - df_python.loc[positive_strand, 'end'].shift(1) - 1

# for negative strand: intron_length = nth start - (n+1)th end
df_python.loc[negative_strand, 'intron_length'] = df_python.loc[negative_strand, 'start'] - df_python.loc[negative_strand, 'end'].shift(-1) - 1


In [43]:
# Set the first intron_length of each transcript to zero
# For positive strand: the first exon has the lowest rank, so get the index of first line of each transcript and transfer the correspond intron_length value to 0
Positive_Head_Idx = df_python[positive_strand].groupby('transcript_id').head(1).index
df_python.loc[Positive_Head_Idx, 'intron_length'] = 0

# For negative strand: the first exon has the highest rank, so get the index of last line of each transcript and transfer the correspond intron_length value to 0
Negative_Tail_Idx = df_python[negative_strand].groupby('transcript_id').tail(1).index
df_python.loc[Negative_Tail_Idx, 'intron_length'] = 0


Bonues Points:

In [44]:
# initiate
df_python['l1'] = df_python['start'] - 100
df_python['l2'] = df_python['start'] + 100
df_python['u1'] = df_python['end'] - 100
df_python['u2'] = df_python['end'] + 100

In [45]:
# add 2 new column to indicate if the row is a head or tail, through this col, we can decide if we should update the begin and end coordinate of l and u
df_python['first_row'] = df_python.groupby(['transcript_id', 'strand']).cumcount() == 0
df_python['last_row'] = df_python.groupby(['transcript_id', 'strand']).cumcount(ascending=False) == 0


C:\Users\zhixz\AppData\Local\Temp\ipykernel_22956\3156434663.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_python['first_row'] = df_python.groupby(['transcript_id', 'strand']).cumcount() == 0
C:\Users\zhixz\AppData\Local\Temp\ipykernel_22956\3156434663.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_python['last_row'] = df_python.groupby(['transcript_id', 'strand']).cumcount(ascending=False) == 0


In [46]:
# update the value based on the cols just add
df_python.loc[(df_python['first_row'] & (df_python['strand'] == '+')), 'l1'] = 0
df_python.loc[(df_python['last_row'] & (df_python['strand'] == '+')), 'u2'] = 0
df_python.loc[(df_python['first_row'] & (df_python['strand'] == '-')), 'u2'] = 0
df_python.loc[(df_python['last_row'] & (df_python['strand'] == '-')), 'l1'] = 0



In [47]:
# deal with the exon that has length less than 200
df_python.loc[df_python['width'] < 200, ['l2', 'u1']] = (df_python['start'] + df_python['end']) / 2

C:\Users\zhixz\AppData\Local\Temp\ipykernel_22956\3142029091.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[12667.  12033.5 12203.  ...  7480.  14708.  15989.5]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_python.loc[df_python['width'] < 200, ['l2', 'u1']] = (df_python['start'] + df_python['end']) / 2


In [48]:
# choose the row that strand is positive and intron length less than 200 and do not inclue the beginning intron transcription
target = (df_python['strand'] == '+') & (df_python['intron_length'] > 0) & (df_python['intron_length'] < 200)

# calculate the new result value
new_val = df_python.loc[target, 'l1'] - df_python.loc[target, 'intron_length'] / 2

# update l1. since it's positive trand, current l1 should be updated
df_python.loc[target, 'l1'] = new_val

# since it's positive strand, u2 of previous line should be updated
prev_rows = df_python.index[target] - 1
df_python.loc[prev_rows, 'u2'] = new_val.values

C:\Users\zhixz\AppData\Local\Temp\ipykernel_22956\2971026104.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.20180000e+04 1.30365000e+04 1.33135000e+04 ... 5.72096775e+07
 5.72115905e+07 5.72115650e+07]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_python.loc[target, 'l1'] = new_val
C:\Users\zhixz\AppData\Local\Temp\ipykernel_22956\2971026104.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.20180000e+04 1.30365000e+04 1.33135000e+04 ... 5.72096775e+07
 5.72115905e+07 5.72115650e+07]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_python.loc[prev_rows, 'u2'] = new_val.values


In [49]:
# choose the row that strand is negative and intron length less than 200 and do not inclue the beginning intron of the transcription
target = (df_python['strand'] == '-') & (df_python['intron_length'] > 0) & (df_python['intron_length'] < 200)

# calculate the new result value
new_val = df_python.loc[target, 'l1'] - df_python.loc[target, 'intron_length'] / 2

# update l1. 
df_python.loc[target, 'l1'] = new_val

# since it's negative strand, u2 of next line should be updated
prev_rows = df_python.index[target] + 1
df_python.loc[prev_rows, 'u2'] = new_val.values